In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

### 데이터 전처리

In [47]:
train_df = pd.read_csv('../data/train.csv', index_col=0)
gen_df = pd.read_csv('../data/gen_data_final.csv', index_col=0)
class_dict = {'협박 대화': 0, '갈취 대화':1, '직장 내 괴롭힘 대화':2, '기타 괴롭힘 대화':3, '일반 대화':4}

In [48]:
train_df

,class,conversation
idx,,
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...


In [49]:
gen_df

,conversation
topic,
오늘 하루 있었던 일,퇴근길이네요! 오늘 하루 수고 많으셨어요.\n네 수고하셨습니다. 특별한 일은 없으셨...
오늘 하루 있었던 일,점심 뭐 드셨어요?\n비빔밥 먹었어요. 맛있더라고요.\n오 저도 비빔밥 좋아하는데!...
오늘 하루 있었던 일,아침에 지하철이 너무 붐벼서 힘들었어요.\n정말요? 저는 버스 탔는데 그것도 만원이...
오늘 하루 있었던 일,날씨가 정말 좋네요. 점심시간에 공원 산책 다녀오셨어요?\n네 잠깐 회사 앞 올림픽...
오늘 하루 있었던 일,오늘 커피 맛이 평소보다 더 좋았던 것 같아요.\n정말요? 저는 오늘따라 커피가 쓰...
...,...
경험/추억,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
경험/추억,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
경험/추억,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...


In [50]:
gen_df['topic'] = '일반 대화'
gen_df = gen_df.rename(columns={'topic':'class'})

In [51]:
data_df = pd.concat([train_df, gen_df], ignore_index=True)

In [52]:
data_df

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
4937,일반 대화,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
4938,일반 대화,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
4939,일반 대화,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...
4940,일반 대화,2002 월드컵 때 거리 응원 나갔던 거 생각하면 아직도 소름 돋아.\n와 맞아! ...


In [53]:
data_df['class'] = data_df['class'].apply(lambda x: class_dict[x])

In [54]:
data_df

,class,conversation
0,0,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,0,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,3,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,1,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
4937,4,너 중학교 때 HOT 팬 아니었어? 방에 브로마이드 붙여놓고 그랬잖아.\n헐 어떻게...
4938,4,대학교 1학년 때 갔던 MT 기억나? 술 마시고 게임하다가 필름 끊겼잖아.\n와 그...
4939,4,나 어릴 때 살던 동네 골목길 사진을 우연히 봤는데 기분이 이상하더라.\n많이 변했...
4940,4,2002 월드컵 때 거리 응원 나갔던 거 생각하면 아직도 소름 돋아.\n와 맞아! ...


In [55]:
# train 데이터의 최대 길이를 구함
data_len = [len(x.split()) for x in data_df['conversation']]
MAX_LEN = max(data_len)
MAX_LEN

456

### 모델

In [45]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from sklearn.model_selection import train_test_split

In [57]:
MODEL_NAME = "klue/bert-base"

In [58]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [60]:
encoded_inputs = tokenizer(
    list(data_df['conversation']),
    padding='max_length', # 또는 padding=True (배치 내 최대 길이에 맞춤)
    truncation=True,
    max_length=MAX_LEN,       # BERT 모델이 처리 가능한 최대 길이 고려 (klue/bert-base는 512)
    return_tensors='tf'
)

In [61]:
labels = tf.constant(data_df['class'].values)
unique_labels = np.unique(labels.numpy())
NUM_CLASSES = len(unique_labels) # 전체 클래스 갯수

In [67]:
labels, unique_labels, NUM_CLASSES

(<tf.Tensor: shape=(4942,), dtype=int64, numpy=array([0, 0, 3, ..., 4, 4, 4])>,
 array([0, 1, 2, 3, 4]),
 5)

In [63]:
num_samples = len(data_df) # 전체 샘플 갯수
indices = np.arange(num_samples) # 인덱스 생성

train_indices, val_indices = train_test_split( # 인덱스를 8대2로 나눔
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels.numpy() # stratify에는 target값으로 class 비율 일정하게 셔플
)

In [64]:
train_inputs = {key: tf.gather(val, train_indices) for key, val in encoded_inputs.items()}
val_inputs = {key: tf.gather(val, val_indices) for key, val in encoded_inputs.items()}

# 레이블도 동일한 인덱스로 선택
train_labels = tf.gather(labels, train_indices)
val_labels = tf.gather(labels, val_indices)

In [65]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels))
train_dataset = train_dataset.shuffle(len(train_indices)).batch(16) # 셔플 및 배치

# 예시: 검증 데이터셋 생성
val_dataset = tf.data.Dataset.from_tensor_slices((val_inputs, val_labels))
val_dataset = val_dataset.batch(16) # 검증 데이터는 보통 셔플하지 않음

In [68]:
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=NUM_CLASSES, from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

In [ ]:
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_cb = EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
    patience=2)

model_checkpoint_cb = ModelCheckpoint(
    filepath='bert_pretrained.h5', # 저장 경로
    monitor='val_loss',           # 관찰 대상: 검증 손실
    save_best_only=True,          # 최고의 모델만 저장
    save_weights_only=True,       # 가중치만 저장
    mode='min',                   # val_loss는 낮을수록 좋으므로 'min'
    verbose=1                     # 저장 시 로그 출력
)

In [ ]:
NUM_EPOCHS = 50
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=NUM_EPOCHS, # 충분한 에폭 수 지정 (조기 종료가 관리)
    callbacks=[early_stopping_cb, model_checkpoint_cb] # 정의된 콜백 전달
)